In [1]:
import os

import geopandas as gpd
import lightning
import optuna
import torch
from lightning import Trainer
from lightning.pytorch.loggers import TensorBoardLogger
from terratorch.tasks.tiled_inference import TiledInferenceParameters

from implementation.training.utils import (
    TrainingTaskHyperparameterTuningConfig,
    create_model,
)
from roofsense.bag3d import BAG3DTileStore
from roofsense.inference.tile import TiledInferenceEngine
from roofsense.preprocessing.parsers.image import ImageParser
from roofsense.preprocessing.parsers.lidar import LiDARParser
from roofsense.preprocessing.stack import RasterStackBuilder
from roofsense.training.datamodule import TrainingDataModule
from roofsense.training.task import TrainingTask

C:\Documents\RoofSense\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.5 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


In [2]:
study_name = "optim"
optim_log_dirpath = os.path.join(r"C:\Documents\RoofSense\logs\3dgeoinfo", study_name)

study = optuna.load_study(
    study_name="optim", storage=f"sqlite:///{optim_log_dirpath}/storage.db"
)

best_params = study.best_params
# Convert parameter format.
for param in ["lab", "tgi"]:
    best_params[f"append_{param}"] = best_params.pop(param)

config = TrainingTaskHyperparameterTuningConfig(
    # Add constant settings.
    # Encoder
    encoder="tu-resnet18d",
    zero_init_last=True,
    # Loss
    label_smoothing=0.1,
    # Optimizer
    optimizer="AdamW",
    # LR Scheduler
    lr_scheduler="CosineAnnealingLR",
    **best_params,
)

In [3]:
# Quantitative Performance Evaluation
torch.set_float32_matmul_precision("high")
torch.backends.cudnn.allow_tf32 = True

lightning.pytorch.seed_everything(0, workers=True)

model = create_model(config)
model_path = os.path.join(
    optim_log_dirpath, f"version_{study.best_trial.number}", "ckpts", "best.ckpt"
)

task: TrainingTask = TrainingTask.load_from_checkpoint(
    checkpoint_path=model_path, model=model
)

datamodule = TrainingDataModule(
    root=r"C:\Documents\RoofSense\roofsense\dataset",
    append_lab=config.append_lab,
    append_tgi=config.append_tgi,
)

test_log_dirpath = os.path.join(os.path.dirname(optim_log_dirpath), "test")
trainer = Trainer(
    logger=TensorBoardLogger(save_dir=test_log_dirpath, name="quantitative"),
    benchmark=True,
)
trainer.test(task, datamodule=datamodule)

Seed set to 0
INFO:timm.models._builder:Loading pretrained weights from Hugging Face hub (timm/resnet18d.ra2_in1k)
INFO:timm.models._hub:[timm/resnet18d.ra2_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
INFO:timm.models._builder:Converted input conv conv1.0 pretrained weights from 3 to 10 channel(s)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 6/6 [00:07<00:00,  0.76it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│       tst/Accuracy0       │            0.0            │
│       tst/Accuracy1       │    0.9483569860458374     │
│       tst/Accuracy2       │    0.8825421929359436     │
│       tst/Accuracy3       │    0.9835869669914246     │
│       tst/Accuracy4       │     0.920972466468811     │
│       tst/Accuracy5       │    0.5459764003753662     │
│       tst/Accuracy6       │    0.9796268939971924     │
│       tst/Accuracy7       │    0.7572770714759827     │
│       tst/Accuracy8       │    0.7811113595962524     │
│       tst/F1Score0        │            0.0            │
│       tst/F1Score1        │     0.926110029220581     │
│       tst/F1Score2        │    0.9206629991531372     │
│       tst/F1Score3        │     0.959437906742096     │
│       tst/F1Score4        │    0.9284178018569946     │
│       tst/F1Score5        │    0.5603682398796082     │
│       tst/F1Score6        │    0.9315889477729797     │
│       tst/F1Score7        │    0.7899554371833801     │
│       tst/F1Score8        │    0.7192239165306091     │
│     tst/JaccardIndex0     │            0.0            │
│     tst/JaccardIndex1     │    0.8623881936073303     │
│     tst/JaccardIndex2     │    0.8529893159866333     │
│     tst/JaccardIndex3     │    0.9220381379127502     │
│     tst/JaccardIndex4     │    0.8663990497589111     │
│     tst/JaccardIndex5     │    0.3892441391944885     │
│     tst/JaccardIndex6     │    0.8719386458396912     │
│     tst/JaccardIndex7     │    0.6528316736221313     │
│     tst/JaccardIndex8     │    0.5615532398223877     │
│         tst/Loss          │     2.000196695327759     │
│     tst/MacroAccuracy     │    0.8499312400817871     │
│     tst/MacroF1Score      │     0.841970682144165     │
│   tst/MacroJaccardIndex   │    0.7474228143692017     │
│    tst/MacroPrecision     │     0.836890697479248     │
│      tst/MacroRecall      │    0.8499312400817871     │
│     tst/MicroAccuracy     │     0.911290168762207     │
│     tst/MicroF1Score      │     0.911290168762207     │
│   tst/MicroJaccardIndex   │    0.8371195197105408     │
│    tst/MicroPrecision     │     0.911290168762207     │
│      tst/MicroRecall      │     0.911290168762207     │
│      tst/Precision0       │            0.0            │
│      tst/Precision1       │    0.9048829078674316     │
│      tst/Precision2       │     0.962225615978241     │
│      tst/Precision3       │    0.9364462494850159     │
│      tst/Precision4       │    0.9359844923019409     │
│      tst/Precision5       │    0.5755393505096436     │
│      tst/Precision6       │    0.8880419731140137     │
│      tst/Precision7       │    0.8255812525749207     │
│      tst/Precision8       │    0.6664232611656189     │
│        tst/Recall0        │            0.0            │
│        tst/Recall1        │    0.9483569860458374     │
│        tst/Recall2        │    0.8825421929359436     │
│        tst/Recall3        │    0.9835869669914246     │
│        tst/Recall4        │     0.920972466468811     │
│        tst/Recall5        │    0.5459764003753662     │
│        tst/Recall6        │    0.9796268939971924     │
│        tst/Recall7        │    0.7572770714759827     │
│        tst/Recall8        │    0.7811113595962524     │
└───────────────────────────┴───────────────────────────┘

[{'tst/Loss': 2.000196695327759,
  'tst/MacroAccuracy': 0.8499312400817871,
  'tst/MacroF1Score': 0.841970682144165,
  'tst/MacroJaccardIndex': 0.7474228143692017,
  'tst/MacroPrecision': 0.836890697479248,
  'tst/MacroRecall': 0.8499312400817871,
  'tst/MicroAccuracy': 0.911290168762207,
  'tst/MicroF1Score': 0.911290168762207,
  'tst/MicroJaccardIndex': 0.8371195197105408,
  'tst/MicroPrecision': 0.911290168762207,
  'tst/MicroRecall': 0.911290168762207,
  'tst/Accuracy0': 0.0,
  'tst/Accuracy1': 0.9483569860458374,
  'tst/Accuracy2': 0.8825421929359436,
  'tst/Accuracy3': 0.9835869669914246,
  'tst/Accuracy4': 0.920972466468811,
  'tst/Accuracy5': 0.5459764003753662,
  'tst/Accuracy6': 0.9796268939971924,
  'tst/Accuracy7': 0.7572770714759827,
  'tst/Accuracy8': 0.7811113595962524,
  'tst/F1Score0': 0.0,
  'tst/F1Score1': 0.926110029220581,
  'tst/F1Score2': 0.9206629991531372,
  'tst/F1Score3': 0.959437906742096,
  'tst/F1Score4': 0.9284178018569946,
  'tst/F1Score5': 0.56036823987

In [ ]:
# Qualitative Performance Evaluation
tile_id = "9-284-556"

tile_store = BAG3DTileStore()

tile_store.asset_manifest(
    tile_id,
    image_index=gpd.read_file(
        r"C:\Documents\RoofSense\roofsense\data\index\image\image.gpkg"
    ),
    lidar_index=gpd.read_file(
        r"C:\Documents\RoofSense\roofsense\data\index\lidar\lidar.gpkg"
    ),
).downl(tile_store.dirpath).save(tile_store.dirpath)

# Parse the data.
ImageParser(tile_store).parse(tile_id)
LiDARParser(tile_store).parse(tile_id)

# Create the raster stack.
RasterStackBuilder(tile_store).merge(tile_id)

test_map_dirpath = os.path.join(test_log_dirpath, "qualitative")
os.makedirs(test_map_dirpath, exist_ok=True)

TiledInferenceEngine(
    ckpt_path=model_path, tile_store=BAG3DTileStore(), model=model
).run(
    tile_id=tile_id,
    dst_filepath=os.path.join(test_map_dirpath, f"{tile_id}.map.pixel.tif"),
    params=TiledInferenceParameters(h_crop=512, h_stride=256, w_crop=512, w_stride=256),
)

C:\Documents\RoofSense\roofsense\utilities\file.py:70: UserWarning: The specified path: 'C:/Users/Dimit/.roofsense/2024.02.28\\37EN1_15.LAZ' does not exist in the system. The value of the 'overwrite' flag will be ignored.
  warnings.warn(msg, UserWarning)
Download Progress:   0%|          | 0/1 [00:00<?, ?File/s]2025-03-28 19:05:33,366 WARNING Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='geotiles.citg.tudelft.nl', port=443): Read timed out. (read timeout=3.05)")': /AHN4_T/37EN1_15.LAZ
2025-03-28 19:05:38,312 WARNING Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='geotiles.citg.tudelft.nl', port=443): Read timed out. (read timeout=3.05)")': /AHN4_T/37EN1_15.LAZ
2025-03-28 19:05:43,826 WARNING Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after conne